### Sleep simulations

#### Installation:

In [ ]:
!pip install numpy==1.24.2
!pip tensorflow-macos==2.11.0

#### Imports:

In [ ]:
from sleep_utils import *
from random import shuffle

#### Baselines without sleep phase alternation

Before modelling how differing schedules of REM / NREM sleep stages affect learning, let's just test whether generative replay helps avoid catastrophic forgeting of representations.

The shuffled_baselines() function below can be used to do this. With baseline_type='new' only the new memories are used to train the VAE. With baseline_type='old' only self-generated memories (i.e. samples from the existing VAE) are used to train the VAE. With baseline_type='both' both of the above are.

In [ ]:
def shuffled_baselines(baseline_type='both',
                       use_initial_weights=True, 
                       latent_dim=20, 
                       seed=0, 
                       inverted=True, 
                       lr=0.001,
                       num_new=1000,
                       num_sampled=1000,
                       continue_training=True):

    np.random.seed(seed)
    
    if inverted is True:
        mnist_train, mnist_test, fmnist_train, fmnist_test = prepare_datasets(split_by_digits=False, 
                                                                              split_by_inversion=True)
    else:
        mnist_train, mnist_test, fmnist_train, fmnist_test = prepare_datasets(split_by_digits=True, 
                                                                              split_by_inversion=False)

    if use_initial_weights is False:
        vae = train_mnist_vae(mnist_train, 'mnist', generative_epochs=25, learning_rate=0.001, latent_dim=latent_dim)
        if inverted is True:
            !mv decoder.h5 decoder_inv.h5
            !mv encoder.h5 encoder_inv.h5
        else:
            !mv decoder.h5 decoder_non_inv.h5
            !mv encoder.h5 encoder_non_inv.h5
    else:
        print("Starting with saved weights:")

    encoder, decoder = models_dict['mnist'](latent_dim=latent_dim)
    vae = VAE(encoder, decoder, kl_weighting=1)
    if inverted is True:
        vae.encoder.load_weights("encoder_inv.h5")
        vae.decoder.load_weights("decoder_inv.h5")
    if inverted is False:
        vae.encoder.load_weights("encoder_non_inv.h5")
        vae.decoder.load_weights("decoder_non_inv.h5")
    opt = keras.optimizers.Adam(learning_rate=lr, jit_compile=False)
    vae.compile(optimizer=opt)
    
    m_err, f_err = plot_error_dists(vae, mnist_test, fmnist_test)
    if inverted is True:
        m_err, f_err = plot_error_dists(vae, mnist_test, fmnist_test, f'{baseline_type}_before', 'MNIST', 'Inverted MNIST')
    else:
        m_err, f_err = plot_error_dists(vae, mnist_test, fmnist_test, f'{baseline_type}_before', 'MNIST 0-4', 'MNIST 5-9')
    check_generative_recall(vae, mnist_test, noise_level=0.0)
    
    sampled_digits = [sample_item(vae, latent_dim=latent_dim) for i in range(100)]
    sampled_digits = np.array(sampled_digits)
    show_samples(sampled_digits)
    
    mnist_errors = []
    fmnist_errors = []
    
    mnist_errors.append(np.mean(m_err))
    fmnist_errors.append(np.mean(f_err))

    random_indices = np.random.choice(fmnist_train.shape[0], num_new, replace=False)
    fmnist_subset = fmnist_train[random_indices]
    sampled_digits = [sample_item(vae, latent_dim=latent_dim) for i in range(num_sampled)]

    if baseline_type == 'new':
        train_data = fmnist_subset
    if baseline_type == 'old':
        train_data = np.array(sampled_digits)
    if baseline_type == 'both':
        train_data = sampled_digits + list(fmnist_subset)
        shuffle(train_data)
        train_data = np.array(train_data)
    
    print("Show training samples:")
    show_samples(train_data)
                           
    vae.fit(train_data, epochs=20, verbose=0, batch_size=32, shuffle=True)
    
    if inverted is True:
        m_err, f_err = plot_error_dists(vae, mnist_test, fmnist_test, f'{baseline_type}_{num_new}new_{num_sampled}sampled', 'MNIST', 'Inverted MNIST')
    else:
        m_err, f_err = plot_error_dists(vae, mnist_test, fmnist_test, f'{baseline_type}_{num_new}new_{num_sampled}sampled', 'MNIST 0-4', 'MNIST 5-9')
    mnist_errors.append(np.mean(m_err))
    fmnist_errors.append(np.mean(f_err))

    check_generative_recall(vae, mnist_test, noise_level=0.0)
    check_generative_recall(vae, fmnist_test, noise_level=0.0)
    
    return mnist_errors, fmnist_errors

In [ ]:
shuffled_baselines(baseline_type='new', inverted=True, use_initial_weights=True)
shuffled_baselines(baseline_type='old', inverted=True)
shuffled_baselines(baseline_type='both', inverted=True, num_new=500, num_sampled=500)

In [ ]:
shuffled_baselines(baseline_type='new', inverted=False, use_initial_weights=True)
shuffled_baselines(baseline_type='old', inverted=False)
shuffled_baselines(baseline_type='both', inverted=False, num_new=500, num_sampled=500)

Vary the number of samples:

In [ ]:
all_mnist_errors = {}
all_fmnist_errors = {}

for num_sampled in [0,10,20,25,30,35,40,50,60]:
    all_mnist_errors[num_sampled] = []
    all_fmnist_errors[num_sampled] = []
    for trial in range(3):
        mnist_errors, fmnist_errors = shuffled_baselines(baseline_type='both', 
                                                         inverted=True, 
                                                         num_new=500, 
                                                         num_sampled=num_sampled)
        all_mnist_errors[num_sampled].append(mnist_errors)
        all_fmnist_errors[num_sampled].append(fmnist_errors)


In [ ]:
sample_sizes = sorted(all_mnist_errors.keys())

mnist_means = [np.mean([x[1] for x in all_mnist_errors[s]]) for s in sample_sizes]
mnist_sems = [np.std([x[1] for x in all_mnist_errors[s]]) / np.sqrt(len([x[1] for x in all_mnist_errors[s]])) for s in sample_sizes]

fmnist_means = [np.mean([x[1] for x in all_fmnist_errors[s]]) for s in sample_sizes]
fmnist_sems = [np.std([x[1] for x in all_fmnist_errors[s]]) / np.sqrt(len([x[1] for x in all_mnist_errors[s]])) for s in sample_sizes]

plt.figure(figsize=(5, 3))
plt.plot(sample_sizes, mnist_means, label='MNIST', color='black', marker='o', alpha=0.5)
plt.plot(sample_sizes, fmnist_means, label='Inverted MNIST', color='blue', marker='o', alpha=0.5)

plt.xlabel('Number of self-generated samples')
plt.ylabel('Mean reconstruction error')
plt.xticks(range(0,60,10))
plt.legend()
plt.savefig('number_of_samples.png', bbox_inches='tight')
plt.show()

#### Train initial VAEs

Train initial VAEs to avoid repeating this each time (leave commented out to use the VAE weights provided):

In [ ]:
# seeds = range(0, 1)
# train_with_schedule_multiple_seeds(seeds, 
#                        num_cycles=10, 
#                        start_fraction_rem=0, 
#                        end_fraction_rem=0,
#                        inverted=True,
#                        use_initial_weights=False)

# !mv decoder.h5 decoder_inv.h5
# !mv encoder.h5 encoder_inv.h5

# seeds = range(0, 1)
# train_with_schedule_multiple_seeds(seeds, 
#                        num_cycles=10, 
#                        start_fraction_rem=0, 
#                        end_fraction_rem=0,
#                        inverted=False,
#                        use_initial_weights=False)

# !mv decoder.h5 decoder_non_inv.h5
# !mv encoder.h5 encoder_non_inv.h5

#### Try different schedules

For example, here we vary just the number of cycles (for a fixed total number of epochs of training).

In [ ]:
rem_fraction_pairs = [(0.5,0.5), (0,0), (1,1)]
cycles_values = [50,20,10,5]
lrs = [0.001]

seeds = range(0, 3)

for lr in lrs:
    for num_cycles in cycles_values:
        for (start_fraction_rem, end_fraction_rem) in rem_fraction_pairs:
            train_with_schedule_multiple_seeds(seeds, 
                                               total_eps=50,
                                               num_cycles=25, 
                                               start_fraction_rem=start_fraction_rem, 
                                               end_fraction_rem=end_fraction_rem,
                                               inverted=True,
                                               lr=lr,
                                               num=5,
                                               continue_training=True)
            